In [1]:
#%%
from ns3gym import ns3env
from comet_ml import Experiment, Optimizer
import tqdm
import subprocess
from collections import deque
import numpy as np

from agents.dqn.agent import Agent, Config
from agents.dqn.model import QNetworkTf
from agents.teacher import Teacher, EnvWrapper
from preprocessor import Preprocessor

/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sheila/

In [2]:
#%%
scenario = "basic"

simTime = 60 # seconds
stepTime = 0.01  # seconds
history_length = 300

EPISODE_COUNT = 15
steps_per_ep = int(simTime/stepTime)

sim_args = {
    "simTime": simTime,
    "envStepTime": stepTime,
    "historyLength": history_length,
    "agentType": Agent.TYPE,
    "scenario": "basic",
    "nWifi": 2,
}

print("Steps per episode:", steps_per_ep)

threads_no = 1
env = EnvWrapper(threads_no, **sim_args)

#%%
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

assert ob_space is not None


Steps per episode: 6000
Waiting for simulation script to connect on port: tcp://localhost:28051
Please start proper ns-3 simulation script using ./waf --run "..."
34.781616|0|0|2|-nan
observation_space dim: 300
Observation space shape: (1, 300)
Action space shape: (1, 1)


In [ ]:
#%%
teacher = Teacher(env, 1, Preprocessor(False))

lr = 4e-4

agent_count = 2 # Number of agents to train
agents = []    # List to store the agents

for i in range(agent_count):
    config = Config(buffer_size=3*steps_per_ep*threads_no, batch_size=32, gamma=0.7, tau=1e-3, lr=lr, update_every=1)
    agent = Agent(QNetworkTf, history_length, action_size=7, config=config)
    agent.set_epsilon(0.9, 0.001, EPISODE_COUNT-2)
    agents.append(agent)

# Test the model
hyperparams = {**config.__dict__, **sim_args}
tags = ["Rew: normalized speed",
        "Final",
        f"{Agent.NAME}",
        sim_args['scenario'],
        f"LR: {lr}",
        f"Instances: {threads_no}",
        f"Station count: {sim_args['nWifi']}",
        *[f"{key}: {sim_args[key]}" for key in list(sim_args)[:3]]]
# # agent.save()
logger = teacher.train(agents, EPISODE_COUNT,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        send_logs=True,
                        experimental=True,
                        tags=tags,
                        parameters=hyperparams)
logger = teacher.eval(agents,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        tags=tags,
                        parameters=hyperparams)
# agent.save()


# %%





The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.dense instead.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





Action space: 7
Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


Action space: 7


COMET INFO: Experiment is live on comet.ml https://www.comet.com/sheila-janota/rl-in-wifi/cf936a799b8c45899d725e8755971ce4



...

100%|██████████| 6300/6300 [09:18<00:00, 11.29it/s, curr_speed=39.54 Mbps, mb_sent=4730.61 Mb] 


Waiting for simulation script to connect on port: tcp://localhost:22971
Please start proper ns-3 simulation script using ./waf --run "..."
Sent 4730.61 Mb/s.	Mean speed: 78.84 Mb/s	Episode 6/15 finished

6
34.781616|0|0|2|-nan


100%|██████████| 6300/6300 [10:24<00:00, 10.08it/s, curr_speed=39.90 Mbps, mb_sent=4732.20 Mb]


Waiting for simulation script to connect on port: tcp://localhost:49546
Please start proper ns-3 simulation script using ./waf --run "..."
Sent 4732.20 Mb/s.	Mean speed: 78.87 Mb/s	Episode 7/15 finished

7
34.781616|0|0|2|-nan


  7%|▋         | 425/6300 [00:32<24:59,  3.92it/s, curr_speed=39.12 Mbps, mb_sent=97.49 Mb]